In [1]:
suppressMessages(library(rwwa))
png_res <- 240

# specify statistical model per variable
mdl_args <- list("rx1day-ondjfm" = list(dist = "gev", type = "fixeddisp", lower = F),
                 "rx1day-djf" = list(dist = "gev", type = "fixeddisp", lower = F),
                 "rx90day" = list(dist = "norm_logt", type = "shift", lower = F))

# standardised return periods per variable & region
return_periods <- list("rx1day-ondjfm_s" = 10, "rx1day-ondjfm_n" = 10,
                       "rx1day-djf_s" = 10, "rx1day-djf_n" = 10,
                       "rx90day_s" = 10, "rx90day_n" = 10)

# load covariates
cov_df <- merge(read.table("ts-obs/gmst.dat", col.names = c("year", "gmst")),
                load_ts("ts-obs/med-storms_nao-djf_era5-stn.dat", col.names = c("year", "nao")))
cov_df$gmst <- cov_df$gmst - cov_df$gmst[cov_df$year == 2026]
cov_df$nao_res <- resid(lm(nao ~ gmst, cov_df))

# define factual & counterfactual climates
cov_2026 <- cov_df[cov_df$year == 2026,c(-1),drop = F]
cov_cf <- rbind("pi" = cov_2026 - c(1.3,0,0),
                "naoneutral" = c(cov_2026$gmst, 0,0),
                "pineutral" = c(-1.3, 0,0))

# covariates to loop over when fitting model variants
cov_list <- list("gmst-only" = "gmst", "gmst+nao" = c("gmst", "nao"), "gmst+naodet" = c("gmst", "nao_res"))

# Modified functions

In [2]:
# revised function to provide results for fixed return period as well as observed event, and to report correlations
mdl_ests <- function (mdl, cov_f, cov_cf, fixed_rp = NA, corrs = F) {
    if (nrow(cov_f) > 1) {
        print("cov_f has more than one row: only first row will be used as factual covariates")
        cov_f <- cov_f[1, , drop = F]
    }
    if (!all(c(sapply(mdl$covnm, function(cnm) cnm %in% colnames(cov_f)), 
        sapply(mdl$covnm, function(cnm) cnm %in% colnames(cov_cf))))) {
        print("Not all model covariates appear in factual/counterfactual covariates: missing covariates will be assumed to be zero throughout")
    }
    pars <- mdl$par
    current_pars <- ns_pars(mdl, fixed_cov = cov_f)
    disp <- current_pars$scale/current_pars$loc

    ev <- mdl$ev
    rp <- return_period(mdl, ev, cov_f)

    changes <- c("PR" = apply(cov_cf, 1, function(cf) prob_ratio(mdl, ev, cov_f, data.frame(t(cf)))),
                 "dI_abs" = apply(cov_cf, 1, function(cf) int_change(mdl, ev, cov_f, data.frame(t(cf)), relative = F)),
                 "dI_rel" = apply(cov_cf, 1, function(cf) int_change(mdl, ev, cov_f, data.frame(t(cf)), relative = T)))

    # results for fixed return period
    if (!is.na(fixed_rp)) {
        res_fixedrp <- unlist(sapply(fixed_rp, function(frp) {
        
            # return level of n-year event
            rl_f <- setNames(eff_return_level(mdl, frp, fixed_cov = cov_f), paste0("rl",frp,"_f"))
            rlcf_f <- apply(cov_cf, 1, function(cf) eff_return_level(mdl, frp, fixed_cov = data.frame(t(cf))))
            rlcf_f <- setNames(rlcf_f, paste0("rl",frp,"_",names(rlcf_f)))
        
            # PR of what is now an n-year event
            pr_f <- apply(cov_cf, 1, function(cf) prob_ratio(mdl, rl_f, cov_f, data.frame(t(cf))))
            pr_f <- setNames(pr_f, paste0("pr",frp,"_",names(pr_f)))
        
            # return period in counterfactual cliamte of what is now an n-year event
            rpcf_f <- apply(cov_cf, 1, function(cf) return_period(mdl, rl_f, data.frame(t(cf))))
            rpcf_f <- setNames(rpcf_f, paste0("rp",frp,"_",names(rpcf_f)))
        
            # # changes in intensity
            # dif_abs <- apply(cov_cf, 1, function(cf) int_change(mdl, rp = frp, cov_f, data.frame(t(cf)), relative = F))
            # dif_rel <- apply(cov_cf, 1, function(cf) int_change(mdl, rp = frp, cov_f, data.frame(t(cf)), relative = T))
                        
            c(rl_f, rlcf_f, pr_f, rpcf_f)
                           
        }, simplify = F))
        changes <- c(changes, res_fixedrp)
    }

    if (corrs) {
        rho <- cor(mdl$data[,c(mdl$varnm, mdl$covnm)])
        rho_names <- sapply(colnames(rho), function(v2) sapply(rownames(rho), function(v1) paste0(v1, "_", v2)))
        rho <- setNames(rho[upper.tri(rho)], paste0("cor_", rho_names[upper.tri(rho)]))
        changes <- c(changes, rho)
    }
    
    if (mdl$dist %in% c("norm_logt")) {
        ev <- exp(ev)
    }
    return(c(mdl$par, disp = disp, event_magnitude = ev, return_period = rp, changes, aic = aic(mdl)))
}

In [3]:
boot_ci <- function (mdl, cov_f, cov_cf, ev, fixed_rp = NA, seed = 42, nsamp = 500, ci = 0.95, corrs = F, return_sample = F) 
{
    alpha <- 1 - ci
    if (missing(ev)) 
        ev <- mdl$ev
    cov_f <- cov_f[, mdl$covnm, drop = F]
    cov_cf <- cov_cf[, mdl$covnm, drop = F]
    if (nrow(cov_f) > 1) {
        print("cov_f has more than one row: only first row will be used as factual covariates")
        cov_f <- cov_f[1, , drop = F]
    }
    for (cnm in mdl$covnm) {
        if (!cnm %in% colnames(cov_f)) {
            cat(cnm, "missing from factual covariates: assumed to be zero\n\n")
            cols_f <- colnames(cov_f)
            cov_f <- cbind(cov_f, 0)
            colnames(cov_f) <- c(cols_f, cnm)
        }
        if (!cnm %in% colnames(cov_cf)) {
            cat(cnm, "missing from counterfactual covariates: assumed to be zero\n\n")
            cols_cf <- colnames(cov_cf)
            cov_cf <- cbind(cov_cf, 0)
            colnames(cov_cf) <- c(cols_cf, cnm)
        }
    }
    obs_res <- mdl_ests(mdl, cov_f, cov_cf, fixed_rp = fixed_rp, corrs = corrs)
    set.seed(seed)
    boot_res <- list()
    i <- 1
    f <- 0
    while (length(boot_res) < nsamp) {
        boot_df <- mdl$data[sample(1:nrow(mdl$data), replace = T), 
            ]
        tryCatch({
            boot_mdl <- refit(mdl, new_data = boot_df)
            boot_res[[i]] <- mdl_ests(boot_mdl, cov_f, cov_cf, fixed_rp = fixed_rp, corrs = corrs)
            i <- i + 1
        }, error = function(cond) {
            f <- f + 1
            return(NULL)
        })
    }
    boot_res <- do.call("cbind", boot_res)
    if (return_sample) {
        return(boot_res)
    }
    else {
        boot_qq <- rbind(t(rbind(est = obs_res, apply(boot_res, 
            1, quantile, c(alpha/2, 1 - (alpha/2)), na.rm = T))), 
            n = c(length(mdl$x), nsamp, f))
        return(boot_qq)
    }
}

# Trend fitting

In [4]:
nsamp <- 1000

for (varnm in c("rx1day-djf", "rx1day-ondjfm", "rx90day")[1]) {
    for (rnm in c("s", "n")) {
        
        for (ds_nm in c("chirps", "cpc", "eobs", "era5", "mswep")) {
            ts <- load_ts(paste0("ts-obs/med-storms_",varnm,"_",rnm,"_",ds_nm,".dat"), col.names = c("year", "x"))
            df <- merge(cov_df, ts)
            rp <- return_periods[[paste0(varnm,"_",rnm)]]

            for (mtype in names(cov_list)) {

                covnm <- cov_list[[mtype]]

                res_fnm = paste0("res-obs/res-obs_",varnm,"_",rnm,"_",ds_nm,"_",mtype,".csv")
                if (file.exists(res_fnm)) next

                mdl <- do.call(fit_ns, append(mdl_args[[varnm]], list("data" = df, "varnm" = "x", covnm = covnm)))

                # updated boot_res function to give correlations and results for a fixed return period
                if (!file.exists(res_fnm)) {
                    boot_res <- boot_ci(mdl, cov_f = cov_2026, cov_cf = cov_cf, nsamp = nsamp, fixed_rp = rp, corrs = T)
                    write.csv(boot_res, res_fnm)
                }
                
                # quick trend plot
                plot_fnm <- gsub("csv", "png", gsub("res-obs/res-obs", "fig-obs/trendplots", res_fnm))
                if (!file.exists(plot_fnm)) {
                    nc <- 1 + length(covnm) * 2
                    png(plot_fnm, h = png_res, w = nc * png_res*1.25); {
                        prep_window(c(1,nc), oma = c(0,0,3,0))
                        plot_trend(mdl)
                        plot_covtrend(mdl, "gmst")
                        if (nc == 5) { plot_covtrend(mdl, covnm[2]) }
                        plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["pi",,drop = F], nsamp = 50, legend_pos = "bottomright")
                        if (nc == 5) { plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["naoneutral",,drop = F], nsamp = 50, legend_pos = "bottomright") }
                        mtext(paste0(varnm, " ~ ",paste0(covnm, collapse = " + "),": ",toupper(ds_nm)," (",rnm,")"), side = 3, outer = T, font = 2, line = -1)
                    }; dev.off()
                }
            }
        }
    }
}

## Compile obs

In [9]:
varnm <- "rx1day-djf"
rnm <- "s"

fl <- list.files("res-obs", pattern = paste0(c(varnm, rnm), collapse = "_"), full.names = T)

res_obs <- plyr::rbind.fill(t(sapply(fl, function(fnm) {
    res <- read.csv(fnm, row.names = "X")
    data.frame(t(unlist(lapply(rownames(res), function(cnm) setNames(res[cnm,], paste(gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))))
}, simplify = F)))

rownames(res_obs) <- gsub(".csv","",apply(sapply(strsplit(fl, "_"), "[", 4:5), 2, paste0, collapse = "_"))
write.csv(res_obs, paste0("res-obs_",varnm,"_",rnm,".csv"))